In [ ]:

"""
    Github Link
        https://github.com/kasrasamareh

    Dataset Link
        https://www.kaggle.com/datasets/samyarr/iranvehicleplatedataset/download?datasetVersionNumber=1
"""

In [ ]:
import os

ROOT_DIR = os.getcwd()+'/'
img_dir = ROOT_DIR+'images/'
xml_dir = ROOT_DIR+'xml/'
test_folder=img_dir+'test/'
train_folder=img_dir+'train/'
csv_file = ROOT_DIR+'file.csv'

--- Plate Detection ---

In [21]:
import os
import glob
import xml.etree.ElementTree as ET

def convert_xml_to_yolo(xml_path, classes):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    image_width = int(root.find('size/width').text)
    image_height = int(root.find('size/height').text)

    yolo_lines = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_name = class_name.lower()
        class_index = classes.index(class_name) if classes.index(class_name)==0 else 0

        bbox = obj.find('bndbox')
        x_min = float(bbox.find('xmin').text)
        y_min = float(bbox.find('ymin').text)
        x_max = float(bbox.find('xmax').text)
        y_max = float(bbox.find('ymax').text)

        x_center = (x_min + x_max) / (2.0 * image_width)
        y_center = (y_min + y_max) / (2.0 * image_height)
        bbox_width = (x_max - x_min) / image_width
        bbox_height = (y_max - y_min) / image_height

        yolo_line = f"{class_index} {x_center} {y_center} {bbox_width} {bbox_height}"
        yolo_lines.append(yolo_line)

    return yolo_lines

def convert_dataset_to_yolo(xml_dir, output_dir, classes):
    os.makedirs(output_dir, exist_ok=True)

    xml_files = glob.glob(os.path.join(xml_dir, '*.xml'))
    for xml_file in xml_files:
        xml_filename = os.path.basename(xml_file)
        txt_filename = os.path.splitext(xml_filename)[0] + ".txt"
        txt_path = os.path.join(output_dir, txt_filename)

        yolo_lines = convert_xml_to_yolo(xml_file, classes)
        with open(txt_path, 'w') as f:
            f.write('\n'.join(yolo_lines))

In [ ]:
convert_dataset_to_yolo(xml_dir, (ROOT_DIR+'labels'), ['licence','vehicle plate'])

In [ ]:
import os
import random
import shutil

def move_files(files, src_dir, dest_dir, split_name):
    split_image_dir = os.path.join(dest_dir, 'images', split_name)
    split_label_dir = os.path.join(dest_dir, 'labels', split_name)
    os.makedirs(split_image_dir, exist_ok=True)
    os.makedirs(split_label_dir, exist_ok=True)

    for file in files:
        src_image = os.path.join(src_dir, 'images', file)
        dest_image = os.path.join(split_image_dir, file)
        shutil.move(src_image, dest_image)

        file_name = os.path.splitext(file)[0]
        txt_file = file_name + '.txt'
        src_txt = os.path.join(src_dir, 'labels', txt_file)
        dest_txt = os.path.join(split_label_dir, txt_file)
        if os.path.exists(src_txt):
            shutil.move(src_txt, dest_txt)

def split_dataset(dataset_dir, output_dir, train_ratio, val_ratio, test_ratio):
    os.makedirs(output_dir, exist_ok=True)

    os.makedirs(os.path.join(dataset_dir, 'images'), exist_ok=True)
    image_dir = os.path.join(dataset_dir, 'images')
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.JPG', '.png'))]

    random.shuffle(image_files)

    num_images = len(image_files)
    num_train = int(train_ratio * num_images)
    num_val = int(val_ratio * num_images)
    num_test = int(test_ratio * num_images)

    train_files = image_files[:num_train]
    val_files = image_files[num_train:num_train + num_val]
    test_files = image_files[num_train + num_val:]

    move_files(train_files, dataset_dir, output_dir, 'train')
    move_files(val_files, dataset_dir, output_dir, 'val')
    move_files(test_files, dataset_dir, output_dir, 'test')

In [ ]:
dataset_dir = ROOT_DIR
output_dir = ROOT_DIR
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

split_dataset(dataset_dir, output_dir, train_ratio, val_ratio, test_ratio)

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
import os

from ultralytics import YOLO


model = YOLO("yolov8n.pt")
model.to(device)
results = model.train(data=ROOT_DIR+"configs.yaml", epochs=10, optimizer='SGD', batch=4, lr0=0.01, cos_lr=True)  # train the model

--- YOLOv8 best model plate detector ---

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
model = YOLO(ROOT_DIR+'runs/detect/train/weights/best.pt')  # build a new model from scratch
model.to(device)

In [ ]:
img = Image.open(ROOT_DIR+'images/test/205.png')
results = model(img)

In [ ]:
from PIL import Image

for detection in results:
    x1, y1, x2, y2 = detection.boxes.boxes.tolist()[0][:4]
    roi = img.crop((x1, y1, x2, y2))

--- OCR ---

-- easyocr --

In [ ]:
import easyocr
from PIL import Image

In [ ]:
img_ = roi

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img_)
ax.axis('off')
plt.show()

In [ ]:
import numpy as np
img_np = np.array(img_)
reader = easyocr.Reader(['fa'], gpu = True)
results = reader.readtext(img_np)

In [ ]:
for detection in results : 
    print("-------")
    print(detection[1])

<!-- -- keras -- -->